In [ ]:
import os
from models.file import File, FileType
from service.embedding import EmbeddingService
from termcolor import colored

PINECONE_INDEX = os.getenv("PINECONE_INDEX", "")
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY", "")
PINECONE_HOST = os.getenv("PINECONE_HOST", "")

file = File(
    type=FileType.pdf,
    url="https://arxiv.org/pdf/2402.05131.pdf"
)
vector_credentials = {
        "type": "pinecone",
        "config": {
            "api_key": PINECONE_API_KEY,
            "host": PINECONE_HOST,
        }
    },

embedding_service = EmbeddingService(
        files=[file],
        index_name=PINECONE_INDEX,
        vector_credentials=vector_credentials
    )

In [ ]:
docs = await embedding_service.generate_documents()

In [ ]:
# Concatenate all the documents
doc_texts = [doc.text for doc in docs]
concatenated_text = " ".join([doc.text for doc in docs])

In [ ]:
from nltk.tokenize import sent_tokenize
nltk_sentences = [sent_tokenize(concatenated_text)]
flattened_nltk_sentences = [sentence for sublist in nltk_sentences for sentence in sublist]

In [ ]:
from semantic_router.utils import splitters
from semantic_router.encoders.openai import OpenAIEncoder

encoder = OpenAIEncoder()

threshold = 0.68  # Adjust this value based on your needs

# Apply the semantic splitter
# takes around 4 minutes
semantic_splits = splitters.semantic_splitter(
    encoder=encoder,
    docs=flattened_nltk_sentences,
    threshold=threshold,
    split_method="cumulative_similarity_drop"
)

In [ ]:
flattened_semantic_splits = [doc for split in semantic_splits for doc in split.docs]
flattened_semantic_splits

In [ ]:
naive_chunks = await embedding_service.generate_chunks(docs)
naive_chunks_text = [chunk.text for chunk in naive_chunks]
naive_chunks_text

In [43]:
def print_colored_chunks(chunks):
    colors = ['red', 'green', 'yellow', 'blue', 'magenta', 'cyan', 'white']
    concatenated_document = ""

    # Check if the first element is a list (indicating a list of lists)
    # If so, flatten it. Otherwise, proceed with the assumption it's a flat list.
    if chunks and isinstance(chunks[0], list):
        flat_chunks = [sentence for chunk in chunks for sentence in chunk]
    else:
        flat_chunks = chunks

    for i, chunk in enumerate(flat_chunks):
        color = colors[i % len(colors)]
        # Use HTML span with style for coloring
        colored_text = f'<span style="color: {color};">{chunk}</span>'
        concatenated_document += colored_text + " "

    return concatenated_document

def compare_chunking_strategies_side_by_side(nltk_sentences, naive_chunks, semantic_splits):
    nltk_result = print_colored_chunks(nltk_sentences)
    naive_result = print_colored_chunks(naive_chunks)
    semantic_result = print_colored_chunks(semantic_splits)

    display(HTML(f"""
    <table>
        <tr>
            <th>NLTK Sentences</th>
            <th>Semantic Splits</th>
            <th>Naive Chunks</th>
        </tr>
        <tr>
            <td style="text-align: left;">{nltk_result}</td>
            <td style="text-align: left;">{semantic_result}</td>
            <td style="text-align: left;">{naive_result}</td>
        </tr>
        </tr>
    </table>
    """))

In [44]:
compare_chunking_strategies_side_by_side(nltk_sentences, naive_chunks_text, flattened_semantic_splits)

NLTK Sentences,Semantic Splits,Naive Chunks
